# Scratchwork #1 - Quaternion Integers

In [1]:
from gaussians import Zi, Qi

## Quaternion Integers

In [99]:
import numpy as np
import random as rnd

class Hi():
    
    def __init__(self,
                 a: (int, np.int64) = 1, b: (int, np.int64) = 0,
                 c: (int, np.int64) = 0, d: (int, np.int64) = 0):
        if isinstance(a, (int, np.int64)):
            self.__arr = np.array([a, b, c, d], dtype=np.int64)
        elif isinstance(a, np.ndarray):
            self.__arr = a
    
    @property
    def real(self) -> int:
        return self.__arr[0]
    
    @property
    def imag(self):
        return tuple(self.__arr[1:])
    
    @property
    def array(self):
        return self.__arr
    
    @property
    def conjugate(self):
        a, b, c, d = self.array
        return Hi(a, -b, -c, -d)
    
    @property
    def norm(self):
        tmp = self * self.conjugate
        return int(tmp.real)
    
    def __repr__(self):
        a, b, c, d = self.array
        return f"Hi({a}, {b}, {c}, {d})"
    
    def __str__(self):
        a, b, c, d = self.array
        return f"({a} + {b}i + {c}j + {d}k)"
    
    def __add__(self, other):
        sum = self.array + other.array
        return Hi(sum)
    
    def __sub__(self, other):
        diff = self.array - other.array
        return Hi(diff)
    
    def __mul__(self, other):
        a1, b1, c1, d1 = self.array
        a2, b2, c2, d2 = other.array
        # See https://en.wikipedia.org/wiki/Quaternion#Hamilton_product
        a = a1 * a2 - b1 * b2 - c1 * c2 - d1 * d2
        b = a1 * b2 + b1 * a2 + c1 * d2 - d1 * c2
        c = a1 * c2 - b1 * d2 + c1 * a2 + d1 * b2
        d = a1 * d2 + b1 * c2 - c1 * b2 + d1 * a2
        return Hi(a, b, c, d)
    
    def to_gaussian_ints(self):
        """Convert this quaternion into two Gaussian integers"""
        a, b, c, d = self.array
        return Zi(int(a), int(b)), Zi(int(c), int(d))
    
    @staticmethod
    def mul_as_gaussian_ints(q1, q2):
        """Implements quaternion multiplication by first converting each one into
        two Gaussian integers, Zi, and then multiplying the Zi according to the
        Cayley-Dickson construction
        """
        a, b = q1.to_gaussian_ints()
        c, d = q2.to_gaussian_ints()
        # 
        # (a, b) * (c, d) = (a * c - d.conj * b, d * a + b * c.conj)
        z1 = a * c - d.conjugate * b
        z2 = d * a + b * c.conjugate
        return Hi(z1.real, z1.imag, z2.real, z2.imag)
    
    @staticmethod
    def random(low=-100, high=100):
        return Hi(np.array([rnd.randint(low, high) for _ in range(4)]))

## Random Quaternion Integers:

In [100]:
rnd.seed(10)
quats = [Hi.random() for _ in range(5)]
h1, h2, h3, h4, h5 = quats

In [101]:
h1, h2, h3, h4, h5

(Hi(46, -92, 9, 23),
 Hi(47, -97, -48, 18),
 Hi(25, -29, 67, -59),
 Hi(-92, 33, 25, -17),
 Hi(-81, -37, 90, -8))

## Conjugation and the Norm

In [102]:
h1.conjugate

Hi(46, 92, -9, -23)

In [103]:
h1.norm

11190

## Quaternion Integer Arithmetic

In [104]:
h1 + h2

Hi(93, -189, -39, 41)

In [105]:
h1 - h2

Hi(-1, 5, 57, 5)

In [106]:
h1 * h2

Hi(-6744, -7520, -2360, 7198)

## Convert to Two Gaussian Integers

In [64]:
h1.to_gaussian_ints()

(Zi(46, -92), Zi(9, 23))

## Alternative Form of Multiplication

In [108]:
help(Hi.mul_as_gaussian_ints)

Help on function mul_as_gaussian_ints in module __main__:

mul_as_gaussian_ints(q1, q2)
    Implements quaternion multiplication by first converting each one into
    two Gaussian integers, Zi, and then multiplying the Zi according to the
    Cayley-Dickson construction



In [107]:
Hi.mul_as_gaussian_ints(h1, h2)

Hi(-6744, -7520, -2360, 7198)

In [5]:
def split_array(arr):
    n = len(arr) // 2
    return arr[:n], arr[n:]

In [6]:
h1 = Hi(1, 2, 3, 4)
h1

Hi(1, 2, 3, 4)

In [7]:
split_array(h1.array)

(array([1, 2]), array([3, 4]))

In [8]:
a, b, c, d = h1.array

In [9]:
a, b, c, d

(1, 2, 3, 4)

In [10]:
type(a)

numpy.int64

In [11]:
print(h1)

(1 + 2i + 3j + 4k)


In [12]:
h1.real

1

In [13]:
h1.imag

(2, 3, 4)

In [14]:
Hi()

Hi(1, 0, 0, 0)

In [15]:
h1 = Hi(1, 2, 3, 4)
h2 = Hi(4, 3, 2, 1)

In [16]:
h1

Hi(1, 2, 3, 4)

In [17]:
h2

Hi(4, 3, 2, 1)

In [18]:
print(h1 + h2)

(5 + 5i + 5j + 5k)


In [19]:
print(h1 - h2)

(-3 + -1i + 1j + 3k)


In [20]:
h1 * h1

Hi(-28, 4, 6, 8)